# Clase 42

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2042%20-%20Estructuras%20de%20Datos%20VII.ipynb)

# Requisitos Previos

* Treap
* Lazy propagation
* Segment Tree

# Treap Implícito

Una característica especial que tienen los treap es que podemos considerar los subárboles del mismo como posiciones ordenadas según el *inorder traversal* del árbol, de manera que si quisiéramos remover todo un rango $[l, r]$ y volverlo a colocar en alguna otra posición (manteniendo el orden relativo de sus elementos), entonces nos bastaría con realizar 2 splits y 1 merge.

Sin embargo, la primera observación que haríamos sería que las llaves serían los índices de las posiciones, y estas deben ser inmutables para las condiciones del treap; entonces, ¿Qué deberíamos hacer?

La respuesta es algo bastante simple: No almacenaremos las llaves explícitamente, simplemente las calcularemos cuando las necesitemos en función a las posiciones y el orden relativo del árbol.

Veamos qué sucede en las funciones split y merge:

- Split: Podemos mantener un contador para saber la cantidad de nodos en un subárbol, de manera que sabremos la posición de un nodo $u$ en $O(1)$ si además llevamos una variable que nos diga cuánto nos hemos desplazado desde la primera posición.

- Merge: Simplemente asumiremos que las llaves del subárbol derecho serán mayores o iguales que las del subárbol izquierdo, y ya que estamos trabajando con posiciones consecutivas del *inorder traversal*, las llaves de todos los nodos del árbol derecho estarán desplazados en la cantidad de nodos en el árbol izquierdo.

De esta manera, podemos realizar las operaciones usuales en $O(\log{n})$.

```C++
pair<node*, node*> split(node* t, int x, int add = 0){
	if(t == nullptr){
		return {nullptr, nullptr};
	}
	int key = add + cnt(t -> left) + 1;
	if(key <= x){
		pair<node*, node*> p = split(t -> right, x, key);
		t -> right = p.first;
		update(t);
		return {t, p.second};
	}
	else{
		pair<node*, node*> p = split(t -> left, x, add);
		t -> left = p.second;
		update(t);
		return {p.first, t};
	}
}
```

En este caso, $add$ será la variable que almacene cuántas posiciones fuera del subárbol son estrictamente menores que la posición del nodo actual, así que deberemos agregarle la cantidad de nodos de su subárbol izquierdo y 1 extra para considerarlo indexado desde $1$.

```C++
node* merge(node* l, node* r){
	if(l == nullptr) return r;
	if(r == nullptr) return l;
	if(l -> priority > r -> priority){
		l -> right = merge(l -> right, r);
		update(l);
		return l;
	}
	else{
		r -> left = merge(l, r -> left);
		update(r);
		return r;
	}
}
```

Por otro lado, la función $merge$ no cambia su estructura, pues no manipula las llaves de los nodos (solo asume su naturaleza), solo sus prioridades.

## Problemas para implementar

- [Array and simple queries](https://www.hackerrank.com/challenges/array-and-simple-queries/problem)
- [Shandom Ruffle](https://codeforces.com/gym/102787/problem/A)
- [Pear TreaP](https://codeforces.com/gym/102787/problem/B)

# Lazy propagation

Para aplicar lazy propagation sobre treaps, es importante notar que al ser un árbol binario, se pueden aplicar las modificaciones como si fuera un Segment Tree, pero estas modificaciones deben **mantener la estructura de Binary Search Tree** entre los nodos.

Lo único que cambia en cada una de las funciones es que, antes de ejecutar alguna acción sobre un nodo no nulo, debemos usar la función push que ya conocemos.

La complejidad total será de $O(\log{n} \cdot upd)$, donde $upd$ es la complejidad de realizar una modificación puntual sobre un nodo. Esto se da debido a que, durante la ejecución del split, tendremos que realizar push sobre $O(\log{n})$ nodos para aislar el rango que queremos, para finalmente actualizar la raíz del treap de dicho rango aislado.

Para un treap implícito, las funciones serían la siguientes:

```C++
pair<node*, node*> split(node* t, int x, int add = 0){
	if(t == nullptr){
		return {nullptr, nullptr};
	}
    push(t);
	int key = add + cnt(t -> left) + 1;
	if(key <= x){
		pair<node*, node*> p = split(t -> right, x, key);
		t -> right = p.first;
		update(t);
		return {t, p.second};
	}
	else{
		pair<node*, node*> p = split(t -> left, x, add);
		t -> left = p.second;
		update(t);
		return {p.first, t};
	}
}
```

```C++
node* merge(node* l, node* r){
	if(l == nullptr) return r;
	if(r == nullptr) return l;
    push(l);
    push(r);
	if(l -> priority > r -> priority){
		l -> right = merge(l -> right, r);
		update(l);
		return l;
	}
	else{
		r -> left = merge(l, r -> left);
		update(r);
		return r;
	}
}
```

## Problemas para implementar

- [Sneetches and Speeches 1](https://codeforces.com/gym/102787/problem/Y)
- [Sneetches and Speeches 3](https://codeforces.com/gym/102787/problem/C)

# Descomposiciones de árboles - I

Hasta ahora habíamos considerado el Euler Tour Representation para realizar tanto consultas como modificaciones a los nodos de un subárbol; sin embargo, también podrían darnos modificaciones pero sobre los elementos del camino entre un par de nodos. ¿Qué haríamos?

Existen dos descomposiciones conocidas, ambas funcionan para la mayoría de los casos, pero la que veremos a continuación es la más general, pues permite combinar la descomposición con otras estructuras de datos para volverse una estructura más potente.

## Heavy-Light Decomposition

Esta descomposición define 2 tipos de arista: pesada (heavy) y liviana (light), de manera que concatenaremos las pesadas en caminos y las livianas estarán libres. Siempre se define el significado de arista pesada y las livianas son las aristas restantes.

![](https://raw.githubusercontent.com/e-maxx-eng/e-maxx-eng/master/img/hld.png)

Hay 2 definiciones de ambos tipos de arista, lo cual no cambia la complejidad de las operaciones pero sí la estructura de los caminos. Veremos ambas y analizaremos sus complejidades respectivas.

### Mayor que la mitad

- **Arista pesada:** Aquella arista $(u, v)$ tal que el tamaño del subárbol de $v$ es mayor que la mitad el tamaño del subárbol de $u$. ($2 subtree[v] > subtree[u]$).

### Mayor que sus hermanos

- **Arista pesada:** Aquella arista $(u, v)$ tal que el tamaño del subárbol de $v$ es mayor que los tamaños de los subárboles de sus hermanos $w$, si hay algún empate, se considera solo una como pesada. ($subtree[v] \geq subtree[w], \forall w \in children[u]\backslash\{v\}$).

**Proposición:** En cualquiera de las dos definiciones, al bajar por una arista liviana $(u, v)$ se cumple que $subtree[v] \leq \frac{subtree[u]}{2}$.

**Prueba:**

1) Mayor que la mitad: Una arista liviana está definida como $(u, v)$ tal que $2 subtree[v] \leq subtree[u] \rightarrow subtree[v] \leq \frac{subtree[u]}{2}$.

2) Mayor que sus hermanos: Una arista liviana $(u, v)$ existe si $u$ tiene al menos 2 hijos, por lo tanto $subtree[v] \leq subtree[w]$, donde $w$ es el hijo de la arista pesada $(u, w)$. Entonces, tendremos que:

$$ subtree[v] \leq subtree[w] \rightarrow 2 subtree[v] \leq subtree[v] + subtree[w] \leq subtree[u] - 1 < subtree[u] $$

$$ 2 subtree[v] < subtree[u] $$

$$ subtree[v] < \frac{subtree[u]}{2} $$

Gracias a la prueba anterior, podemos deducir que para llegar desde cualquier nodo $x$ hasta la raíz del árbol tendremos que pasar por un máximo de $O(\log{n})$ aristas livianas y, como consecuencia, por $O(\log{n})$ **caminos de aristas pesadas**.

Si usamos alguna estructura logarítmica (Segment Tree suele ser el más usado) para almacenar la información de los caminos de aristas pesadas, tendremos una complejidad de $O(\log^{2}{n})$ tanto para consulta como para modificación.

La implementación siguiente es de la definición "Mayor que sus hermanos", esta realiza la descomposición luego de un preprocesamiento de los tamaños de subárboles.

```C++
int T = 0;

void DFS(int u, int p = -1){
    subtree[u] = 1;
    for(int i = 0; i < G[u].size(); i++){
        if(G[u][i] == p){
            swap(G[u].back(), G[u][i]);
        }
        int v = G[u][i];
        if(v == p) continue;
        DFS(v, u);
        if(subtree[v] > subtree[G[u][0]]){
            swap(G[u][i], G[u][0]);
        }
        subtree[u] += subtree[v];
    }
    if(p != -1){
        G[u].pop_back();
    }
}

void HLD(int u){
    in[u] = T++;
    for(int v : G[u]){
        par[v] = u;
        nxt[v] = (v == G[u][0] ? nxt[u] : v);
        h[v] = h[u] + 1;
        HLD(v);
    }
    out[u] = T - 1;
}
```

Notemos que estaremos guardando algunos arreglos extra:

- $in$ y $out$ son los timestamps de entrada y salida según el Euler Tour.

- $nxt[u]$ es el inicio de la cadena de $u$ (si el nodo pertenece a una arista liviana, $nxt[u] = u$).

- $h$ son las profundidades de cada nodo.

**Propiedad:** El rango $[in[nxt[u]], in[u]]$ contiene todos los nodos desde el inicio de la cadena del nodo $u$ hasta si mismo. Esto se da porque estamos colocando al nodo de la arista pesada como primero en la lista de adyacencia.

### Consultas

Para consultar información sobre un camino $u \leadsto v$ uno pensaría en obtener el LCA $l$ de ambos nodos y luego obtener las respuestas parciales de cada subcamino $u \leadsto l$ y $v \leadsto l$; sin embargo, es más sencillo hacer la siguiente observación:

**Observación:** El LCA de $u$ y $v$ pertenece a una sola cadena, a la cual llegarán eventualmente los nodos $u$ y $v$ a medida que asciendan en el árbol. Además, esta cadena es la más alta a la que podrán llegar sin salirse del camino $u \leadsto v$.

Gracias a la observación anterior, podemos considerar el mover hacia arriba a cada nodo hasta que estos estén en la misma cadena (fácilmente identificable, pues esto se dará cuando $nxt[u] = nxt[v]$). Entonces, podemos aplicar el algoritmo greedy de subir lo más que se pueda a aquel que tenga $nxt$ con menor $h$.

La siguiente implementación está hecha para consultar el máximo valor de algún nodo en el camino $a \leadsto b$, dado que tenemos un segment tree sobre el Euler Tour del árbol.

```C++
int path(int a, int b){
	int res = -2e9;
	while(nxt[a] != nxt[b]){
		if(h[nxt[a]] < h[nxt[b]]) swap(a, b);
		res = max(res, query(in[nxt[a]], in[a]));
		a = par[nxt[a]];
	}
	if(h[a] > h[b]) swap(a, b);
	res = max(res, query(in[a], in[b]));
	return res;
}
```

Es sencillo notar que el bucle se ejecutará hasta que estén ambos nodos en la misma cadena, para lo cual realizamos una consulta extra para cubrir dicho rango de nodos. La complejidad de este algoritmo es de $O(\log^{2}{n})$.

### Modificaciones

Para realizar modificaciones sobre un camino podemos considerar el mismo algoritmo de consulta pero aplicando las actualizaciones:

```C++
void changePath(int a, int b, int w){
	while(nxt[a] != nxt[b]){
		if(h[nxt[a]] < h[nxt[b]]) swap(a, b);
		update(in[nxt[a]], in[a], w);
		a = par[nxt[a]];
	}
	if(h[a] > h[b]) swap(a, b);
    update(in[a], in[b], w);
	return res;
}
```

Cuya complejidad será la misma de $O(\log^{2}{n})$. Por otro lado, si queremos modificar sobre un subárbol, podemos usar el mismo Segment Tree (si se trabaja sobre las mismas propiedades) y modificar el rango $[in[u], out[u]]$. Este segundo tipo de modificación tomará $O(\log{n})$.

Podemos notar que esta forma de implementación es bastante flexible y potente.

**Nota:** Si las modificaciones e información son sobre las aristas y no sobre los nodos, podemos considerar que la posición del nodo $u$ representa a la arista $(par[u], u)$ y realizar las modificaciones correspondientes a las funciones.

### Problema para implementar

- [Query on a tree](https://www.spoj.com/problems/QTREE/)
- [Subtrees And Paths](https://www.hackerrank.com/challenges/subtrees-and-paths/problem)
- [Can you answer these queries VII](https://www.spoj.com/problems/GSS7/)
- [Queries on tree again!](https://www.codechef.com/problems/QTREE)